## REST API for DataMart

This notebook showcases how to use the Rest API for the DataMart system.

For the augmentation, we use the taxi demand example, available here: https://gitlab.datadrivendiscovery.org/d3m/datasets/tree/master/seed_datasets_data_augmentation/DA_ny_taxi_demand

The documentation for the REST API is available here: https://auctus.vida-nyu.org/swagger/#/

In [1]:
from io import BytesIO
import json
import os
import pandas as pd
from pprint import pprint
from pathlib import Path
import requests
import zipfile

In [2]:
def print_results(results):
    if not results:
        return
    for result in results:
        print(result['metadata']['name'])
        print('Score: ', result['score'])
        if 'augmentation' in result:
            aug_type = result['augmentation']['type']
            print('Augmentation: %s' % aug_type)
            print("Left Columns: %s" %
                  str(result['augmentation']['left_columns_names']))
            print("Right Columns: %s" %
                  str(result['augmentation']['right_columns_names']))
            
        print("-------------------")

Initially, we have the taxi demand data.

In [3]:
# You can change this accordingly
ny_taxi_demand_dir = os.path.expanduser('~/projects/d3m/datasets/seed_datasets_data_augmentation/DA_ny_taxi_demand/DA_ny_taxi_demand_dataset')
ny_taxi_demand_file = os.path.join(ny_taxi_demand_dir, 'tables', 'learningData.csv')
ny_taxi_demand_table = pd.read_csv(ny_taxi_demand_file)

In [4]:
ny_taxi_demand_table.head()

,d3mIndex,tpep_pickup_datetime,num_pickups
0,0,2018-04-19 22:00:00,731
1,1,2018-06-30 20:00:00,183
2,2,2018-06-02 10:00:00,384
3,3,2018-04-17 13:00:00,648
4,4,2018-01-04 01:00:00,3


### Searching for Datasets

Let's use DataMart to search for a weather datasets that can be used to augment the taxi demand one.

In [5]:
URL = 'https://auctus.vida-nyu.org'

In [6]:
url = URL + '/search'
query = {
    'keywords': ['transportation', 'city data', 'taxi',
                 'yellow cab', 'pickup', 'LaGuardia airport',
                 'weather', 'weather conditions', 'new york', 'hourly']
}

# http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file
with open(ny_taxi_demand_file, 'rb') as data:
    response = requests.post(
        url,
        files={
            'data': data,
            'query': ('query.json', json.dumps(query), 'application/json'),
        }
    )
if response.status_code == 400:
    try:
        print('Error: %s' % response.json()['error'])
    except (KeyError, ValueError):
        pass
response.raise_for_status()
query_results = response.json()['results']

In [7]:
print_results(query_results)

Newyork Weather Data around Airport 2016-18
Score:  50.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['DATE']]
-------------------
Traffic Crashes - Crashes
Score:  40.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['CRASH_DATE']]
-------------------
Traffic Crashes - Crashes
Score:  40.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['DATE_POLICE_NOTIFIED']]
-------------------
Fare Card History for Metropolitan Transportation Authority (MTA): Beginning 2010
Score:  30.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['From Date']]
-------------------
Fare Card History for Metropolitan Transportation Authority (MTA): Beginning 2010
Score:  30.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['To Date']]
-------------------
311 Service Requests - Pot Holes Reported - Historical
Score:  30.0
Augmentation: join
Left Columns: [['tpep_pickup

We could also just simply search for all of the datasets that can be joined/unioned with the taxi demand one.

In [8]:
url = URL + '/search'

# http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file
with open(ny_taxi_demand_file, 'rb') as data:
    response = requests.post(
        url,
        files={
            'data': data
        }
    )
if response.status_code == 400:
    try:
        print('Error: %s' % response.json()['error'])
    except (KeyError, ValueError):
        pass
response.raise_for_status()
query_results = response.json()['results']

In [9]:
print_results(query_results)

Asotin County Library District Operating Revenues
Score:  1.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['Year']]
-------------------
COPA Cases - By Complainant or Subject
Score:  1.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['COMPLAINT_DATE']]
-------------------
State University Construction Fund (SUCF) Contracts: Beginning 1995
Score:  1.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['Contract Award Date']]
-------------------
State University Construction Fund (SUCF) Contracts: Beginning 1995
Score:  1.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['Contract End Date']]
-------------------
311 Service Requests - Tree Debris - Historical
Score:  1.0
Augmentation: join
Left Columns: [['tpep_pickup_datetime']]
Right Columns: [['Creation Date']]
-------------------
311 Service Requests - Tree Debris - Historical
Score:  1.0
Augmentation: join
Left Column

This shows the importance of also using the query schema to filter results, as there can be many datasets that can be joined or unioned with the input data.

### Downloading a Dataset

Now let's materialize the weather dataset, in case the user wants to take a look at the data before augmenting it (or so that the user can augment the data him/herself).

In [10]:
url = URL + '/search'
query = {
    'keywords': ['transportation', 'city data', 'taxi',
                 'yellow cab', 'pickup', 'LaGuardia airport',
                 'weather', 'weather conditions', 'new york', 'hourly']
}

# http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file
with open(ny_taxi_demand_file, 'rb') as data:
    response = requests.post(
        url,
        files={
            'data': data,
            'query': ('query.json', json.dumps(query), 'application/json'),
        }
    )
if response.status_code == 400:
    try:
        print('Error: %s' % response.json()['error'])
    except (KeyError, ValueError):
        pass
response.raise_for_status()
query_results = response.json()['results']

In [11]:
url = URL + '/download'
id_ = query_results[0]['id']
params = {'format': 'd3m'} # returns a .zip file with the data and its corresponding datasetDoc

response = requests.get(url + '/%s' % id_, params=params)
if response.status_code == 400:
    try:
        print('Error: %s' % response.json()['error'])
    except (KeyError, ValueError):
        pass
response.raise_for_status()

zip_ = zipfile.ZipFile(BytesIO(response.content), 'r')
learning_data = pd.read_csv(zip_.open('tables/learningData.csv'))
dataset_doc = json.load(zip_.open('datasetDoc.json'))
zip_.close()

In [12]:
learning_data.head()

,DATE,HOURLYSKYCONDITIONS,HOURLYDRYBULBTEMPC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure
0,2016-01-01 01:00:00,OVC:08 38,6.1,58.0,17.0,300,30.03
1,2016-01-01 02:00:00,OVC:08 38,6.1,56.0,16.0,320,30.03
2,2016-01-01 03:00:00,OVC:08 38,5.6,55.0,13.0,340,30.03
3,2016-01-01 04:00:00,OVC:08 36,5.6,55.0,13.0,300,30.03
4,2016-01-01 05:00:00,FEW:02 34 OVC:08 45,5.0,60.0,13.0,270,30.01


### Augmenting a Dataset

Let's try to do our augmentation for the first query result.

In [13]:
url = URL + '/augment'
task = query_results[0]

# http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file
with open(ny_taxi_demand_file, 'rb') as data:
    response = requests.post(
        url,
        files={
            'data': data,
            'task': ('task.json', json.dumps(task), 'application/json'),
        },
        stream=True,
    )
if response.status_code == 400:
    try:
        print('Error: %s' % response.json()['error'])
    except (KeyError, ValueError):
        pass
response.raise_for_status()
zip_ = zipfile.ZipFile(BytesIO(response.content), 'r')
learning_data = pd.read_csv(zip_.open('tables/learningData.csv'))
dataset_doc = json.load(zip_.open('datasetDoc.json'))
zip_.close()

In [14]:
learning_data.head()

,d3mIndex,tpep_pickup_datetime,num_pickups,HOURLYSKYCONDITIONS,mean HOURLYDRYBULBTEMPC,sum HOURLYDRYBULBTEMPC,max HOURLYDRYBULBTEMPC,min HOURLYDRYBULBTEMPC,mean HOURLYRelativeHumidity,sum HOURLYRelativeHumidity,max HOURLYRelativeHumidity,min HOURLYRelativeHumidity,mean HOURLYWindSpeed,sum HOURLYWindSpeed,max HOURLYWindSpeed,min HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure
0,0,2018-04-19 22:00:00,731,NaN,5.0,5.0,5.0,5.0,53.0,53.0,53.0,53.0,16.0,16.0,16.0,16.0,NaN,NaN
1,1,2018-06-30 20:00:00,183,NaN,30.6,30.6,30.6,30.6,43.0,43.0,43.0,43.0,5.0,5.0,5.0,5.0,NaN,NaN
2,2,2018-06-02 10:00:00,384,NaN,28.3,28.3,28.3,28.3,61.0,61.0,61.0,61.0,6.0,6.0,6.0,6.0,NaN,NaN
3,3,2018-04-17 13:00:00,648,NaN,8.3,8.3,8.3,8.3,44.0,44.0,44.0,44.0,17.0,17.0,17.0,17.0,NaN,NaN
4,4,2018-01-04 01:00:00,3,NaN,-1.7,-1.7,-1.7,-1.7,45.0,45.0,45.0,45.0,8.0,8.0,8.0,8.0,NaN,NaN


In [14]:
learning_data.head()

,d3mIndex,tpep_pickup_datetime,num_pickups,HOURLYSKYCONDITIONS,HOURLYDRYBULBTEMPC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure
0,0,2018-04-19 22:00:00,731,FEW:02 42,5.0,53.0,16.0,310,29.97
1,1,2018-06-30 20:00:00,183,SCT:04 250,30.6,43.0,5.0,180,29.97
2,2,2018-06-02 10:00:00,384,FEW:02 40 FEW:02 150 SCT:04 200,28.3,61.0,6.0,070,29.70
3,3,2018-04-17 13:00:00,648,BKN:07 46 BKN:07 85,8.3,44.0,17.0,260,29.60
4,4,2018-01-04 01:00:00,3,OVC:08 32,-1.7,45.0,8.0,020,29.91


And we have our augmented data!

DataMart also produces a datasetDoc JSON object for the dataset. However, note that this datasetDoc JSON **does not** preserve the information from the supplied data's datasetDoc JSON. You need to use the Python DataMart API for that: https://gitlab.com/datadrivendiscovery/datamart-api/blob/master/datamart.py

In [15]:
pprint(dataset_doc, indent=2)

{ 'about': { 'approximateSize': '241432 B',
             'datasetID': '5d964508e25a406f8b3b116d99b63533',
             'datasetName': '5d964508e25a406f8b3b116d99b63533',
             'datasetSchemaVersion': '4.0.0',
             'datasetVersion': '1.0',
             'license': 'unknown',
             'redacted': False},
  'dataResources': [ { 'columns': [ { 'colIndex': 0,
                                      'colName': 'd3mIndex',
                                      'colType': 'integer',
                                      'role': ['index']},
                                    { 'colIndex': 1,
                                      'colName': 'tpep_pickup_datetime',
                                      'colType': 'dateTime',
                                      'role': ['attribute']},
                                    { 'colIndex': 2,
                                      'colName': 'num_pickups',
                                      'colType': 'integer',
                    